In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/amansingh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from attractions_recc import *
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

In [3]:
att_df = pd.read_json('etl/attractions.json',orient='records')

In [4]:
att_df = pd.read_json('/Users/amansingh/Desktop/Intelligent-Travel-Recommendation-System-master-1/etl/attractions.json',orient='records')

In [5]:
att_df.head()

,province,category,attraction_id,city,country,name,price,rating,avg_price,latitude,longitude
0,british_columbia,featured_tours_and_tickets,0,vancouver,canada,vancouver_city_sightseeing_tour,80.0,4.5,145.00,49.197834,-123.064995
1,british_columbia,featured_tours_and_tickets,1,vancouver,canada,vancouver_to_victoria_and_butchart_gardens_tou...,210.0,5.0,145.00,49.197834,-123.064995
2,quebec,featured_tours_and_tickets,2,montreal,canada,quebec_city_and_montmorency_falls_day_trip_fro...,115.0,4.5,115.00,45.500145,-73.572029
3,ontario,featured_tours_and_tickets,3,toronto,canada,niagara_falls_day_trip_from_toronto,169.0,5.0,212.21,43.656151,-79.384262
4,ontario,featured_tours_and_tickets,4,niagara_falls,canada,"best_of_niagara_falls_tour_from_niagara_falls,...",158.0,5.0,212.21,43.085712,-79.082428


## User details and Preferences

In [6]:
uname = w.Text(description="User Name")
place = w.Text(value = 'Province',  description="Destination")
budget = w.IntRangeSlider(min=att_df.price.min(), max=att_df.price.max(),step=10,value=[att_df.price.min(),att_df.price.max()],description="Budget")
v1 = w.VBox([uname,place,budget])
start = w.DatePicker(description='Start Date',disabled=False)
end = w.DatePicker(description='End Date',disabled=False)
v2 = w.VBox([start,end])
out = w.HBox([v1,v2])
display(out)

In [7]:
category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)
cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = w.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = w.Layout(width='100%', height='100px')
but_items = [w.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = w.VBox([i for i in but_items[:4]])
c2 = w.VBox([i for i in but_items[4:8]])
c3 = w.VBox([i for i in but_items[8:12]])
c4 = w.VBox([i for i in but_items[12:16]])
c5 = w.VBox([i for i in but_items[16:]])
buttons = w.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

Select and rate atleast 5 categories and rate them:


private_&_custom_tours


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

food,_wine_&_nightlife


IntSlider(value=0, description='Rate', max=5)

...

IntSlider(value=0, description='Rate', max=5)

Rate 2 more!

luxury_&_special_occasions


IntSlider(value=0, description='Rate', max=5)

Rate 1 more!

air,_helicopter_&_balloon_tours


IntSlider(value=0, description='Rate', max=5)

In [8]:
user_name = re.sub(' ','_',uname.value.lower())
province = re.sub(' ','_',place.value.lower())
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value.value)

## Model recommendation and filtering

In [9]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [10]:
filename, user, rbm_att = get_recc(att_df, cat_rating)
with_url = filter_df(filename, user, low, high, province, att_df)

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 787
Model restored


,attraction_id,Recommendation Score
0,0,0.018642
1,1,0.003147
2,2,0.000258
3,3,0.000034
4,4,0.000369


In [11]:
%%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+2):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(with_url, final)
    else:
        final = find_closest(with_url, final['location'][-1],final['timeofday'][i], final)

## Final Output

In [12]:
print(final['image'])

['downloads/niagara_falls_in_one_day:_deluxe_sightseeing_tour_of_american_and_canadian_sides/1. 6-hour-niagara-falls.jpg', 'downloads/niagara_falls_canadian_side_tour_and_maid_of_the_mist_boat_ride/1. niagara-falls-canadian.jpg', 'downloads/6-hour_niagara_falls_canadian_side_evening_illuminations_tour/1. 6-hour-niagara-falls.jpg', 'downloads/epic_toronto_night_tour/1. epic-toronto-night-tour.jpg', 'downloads/best_of_niagara_falls_tour_from_niagara_falls/1. best-of-niagara-falls-tour-from-niagara-falls-ontario-in-niagara-falls-190431.jpg', 'downloads/ultimate_toronto_tour/1. ultimate-toronto-tour.jpg', 'downloads/niagara_falls_night_tour_with_dinner_and_cruise/1. niagara-falls-night-tour-with-dinner-and-cruise-in-niagara-falls-218806.jpg', 'downloads/day_and_night_tour_of_niagara_falls/1. day-and-night-tour-of-niagara-falls-in-niagara-falls-193929.jpg', 'downloads/niagara_falls_helicopter_tour/1. niagara-falls-heliopter_0.jpg', 'downloads/bird_kingdom_all_day_admission_ticket/1. bird-ki

In [13]:
days = (end_date - begin_date).days + 1
display(final_output(days,final))